<p style="font-size: 200%; font-weight: bold; color: teal; padding-bottom: 20px">Scala By Example</p>
Notebook adaptation of a draft by Martin Odersky, Programming Methods Laboratory,  EPFL,  Switzerland

https://www.scala-lang.org/old/sites/default/files/linuxsoft_archives/docu/files/ScalaByExample.pdf


***Use this notebook to familiarize yourself with Scala***

At the same time, Scala is compatible with Java. Java libraries and frameworks can be used without glue code or additional declarations.
This document introduces Scala in an informal way, through a sequence of examples.
Chapters 2 and 3 highlight some of the features that make Scala interesting. The following chapters introduce the language constructs of Scala in a more thorough way, starting with simple expressions and functions, and working up through objects and classes, lists and streams, mutable state, pattern matching to more complete examples that show interesting programming techniques. The present informal exposition is meant to be complemented by the Scala Language Reference Manual which specifies Scala in a more detailed and precise way.



# A First Example
As a first example, here is an implementation of Quicksort in Scala.

In [1]:
def sort(xs: Array[Int]) {
    def swap(i: Int, j: Int) {
        val t = xs(i); xs(i) = xs(j); xs(j) = t
    }

    def sort1(l: Int, r: Int) {
        val pivot = xs((l + r) / 2)
        var i = l; var j = r
        while (i <= j) {
            while (xs(i) < pivot) i += 1
            while (xs(j) > pivot) j -= 1
            if (i <= j) {
                swap(i, j)
                i += 1
                j -= 1
            }
        }
        if (l < j) sort1(l, j)
        if (j < r) sort1(i, r)
    }
    sort1(0, xs.length - 1)
}

In [2]:
// create a mutable(!) array
var x = Array(2, 5, 8, 5, 0, 9)

// print results
for (k <- x) println(k)

2
5
8
5
0
9


In [5]:
var y = Array(2, 5, 8, 5, 0, 9, 7, 4, 3)
y(0)

2

In [3]:
// now sort and print array again
sort(x)
for (k <- x) println(k)

0
2
5
5
8
9


The implementation looks quite similar to what one would write in Java or C. We use the same operators and similar control structures. There are also some minor syntactical differences. In particular:
- Definitions start with a reserved word. Function definitions start with def, variable definitions start with var and definitions of values (i.e. read only variables) start with val.
- The declared type of a symbol is given after the symbol and a colon. The declared type can often be omitted, because the compiler can infer it from the context.
- Array types are written `Array[T]` rather than `T[]`,and array selections are written `a(i)` rather than `a[i]`.
- Functions can be nested inside other functions. Nested functions can access parameters and local variables of enclosing functions. For instance, the name of the array `xs` is visible in functions `swap` and `sort1`, and therefore need not be passed as a parameter to them.

So far, Scala looks like a fairly conventional language with some syntactic peculiarities. In fact it is possible to write programs in a conventional imperative or object-oriented style. This is important because it is one of the things that makes it easy to combine Scala components with components written in mainstream languages such as Java, C# or Visual Basic.


However, it is also possible to write programs in a style which looks completely different. Here is Quicksort again, this time written in functional style.

In [26]:
def sort(xs: Array[Int]): Array[Int] = {
    if (xs.length <= 1) xs
    else {
        val pivot = xs(xs.length / 2)
        Array.concat(
            sort(xs filter (pivot >)),
                 xs filter (pivot ==),
            sort(xs filter (pivot <)))
    }
}


In [27]:
var x = Array(2, 5, 8, 5, 0, 9)
sort(x)
for (k <- x) println(k)

2
5
8
5
0
9


The functional program captures the essence of the quicksort algorithm in a concise way:
- If the array is empty or consists of a single element, it is already sorted, so return it immediately.
- If the array is not empty, pick an element in the middle of it as a pivot.
- Partition the array into two sub-arrays containing elements that are less than, respectively greater than the pivot element, and a third array which contains elements equal to pivot.
- Sort the first two sub-arrays by a recursive invocation of the sort function. (This is not quite what the imperative algorithm does; the latter partitions the array into two sub-arrays containing elements less than or greater or equal to pivot)
- The result is obtained by appending the three sub-arrays together.


Both the imperative and the functional implementation have the same asymptotic complexity – $O(N log(N))$ in the average case and $O(N^2)$ in the worst case. But where the imperative implementation operates in place by modifying the argument array, the functional implementation returns a new sorted array and leaves the argument array unchanged. The functional implementation thus requires more transient memory than the imperative one.

The functional implementation makes it look like Scala is a language that’s specialized for functional operations on arrays. In fact, it is not; all of the operations used in the example are simple library methods of a sequence `class Seq[T]` which is part of the standard Scala library, and which itself is implemented in Scala. Because arrays are instances of Seq all sequence methods are available for them.

In particular, there is the method filter which takes as argument a **predicate function**. This predicate function must map array elements to boolean values. The result of `filter` is an array consisting of all the elements of the original array for which the given predicate function is true. The filter method of an object of type `Array[T]` thus has the signature
`def filter(p: T => Boolean): Array[T]`

Here, `T => Boolean` is the type of functions that take an element of type `T` and return a Boolean.
Functions like filter that take another function as argument or return one as result are called higher-order functions.

Scala does not distinguish between identifiers and operator names. An identifier can be either a sequence of letters and digits which begins with a letter, or it can be a sequence of special characters, such as “+”, “*”, or “:”.

Any identifier can be used as an infix operator in Scala. The binary operation $E\;op\;E'$ is always interpreted as the method call $E.op(E′)$.
This holds also for binary infix operators which start with a letter. Hence, the expression `xs filter (pivot >)` is equivalent to the method call `xs.filter(pivot >)`.

In the quicksort program, filter is applied three times to an anonymous function argument. The first argument, `pivot >`, represents a function that takes an argument `x` and returns the value `pivot > x`. This is an example of a partially applied function. Another, equivalent way to write this function which makes the missing argument explicit is `x => pivot > x`. The function is anonymous, i.e. it is not defined with a name. The type of the x parameter is omitted because a Scala compiler can infer it automatically from the context where the function is used. To summarize, `xs.filter(pivot >)` returns a list consisting of all elements of the list `xs` that are smaller than pivot.
Looking again in detail at the first, imperative implementation of Quicksort, we find that many of the language constructs used in the second solution are also present, albeit in a disguised form.

# Expressions and Simple Functions

## Expressions And Simple Functions

A Scala system comes with an interpreter which can be seen as a fancy calculator. A user interacts with the calculator by typing in expressions. The calculator returns the evaluation results and their types. For example:

    scala> 87 + 145
    unnamed0: Int = 232
    
    scala> 5 + 2 * 3
    unnamed1: Int = 11
    
    scala> "hello" + " world!"
    unnamed2: java.lang.String = hello world!


It is also possible to name a sub-expression and use the name instead of the expression afterwards:

    scala> def scale = 5
    scale: Int
    
    scala> 7 * scale
    unnamed3: Int = 35
    
    scala> def pi = 3.141592653589793
    pi: Double
    
    scala> def radius = 10
    radius: Int
    
    scala> 2 * pi * radius
    unnamed4: Double = 62.83185307179586


Definitions start with the reserved word def; they introduce a name which stands for the expression following the = sign. The interpreter will answer with the introduced name and its type.

Executing a definition such as `def x = e` will not evaluate the expression `e`. Instead e is evaluated whenever `x` is used. Alternatively, Scala offers a value definition `val x = e`, which does evaluate the right-hand-side as part of the evaluation of the definition. If `x` is then used subsequently, it is immediately replaced by the precomputed value of `e`, so that the expression need not be evaluated again.

How are expressions evaluated? An expression consisting of operators and operands is evaluated by repeatedly applying the following simplification steps.
- pick the left-most operation
- evaluate its operands
- apply the operator to the operand values.

A name defined by `def` is evaluated by replacing the name by the (unevaluated) definition's right hand side. A name defined by `val` is evaluated by replacing the name by the value of the definition's right-hand side. The evaluation process stops once we have reached a value. A value is some data item such as a string, a number, an array, or a list.

## Parameters
Using def, one can also define functions with parameters. For example:

    scala> def square(x: Double) = x * x
    
    square: (Double)Double
    scala> square(2)
    unnamed0: Double = 4.0
    
    scala> square(5 + 3)
    unnamed1: Double = 64.0
    
    scala> square(square(4))
    unnamed2: Double = 256.0
    
    scala> def sumOfSquares(x: Double, y: Double) = square(x) + square(y)
    sumOfSquares: (Double,Double)Double
    
    scala> sumOfSquares(3, 2 + 2)
    unnamed3: Double = 25.0

Function parameters follow the function name and are always enclosed in parentheses. Every parameter comes with a type, which is indicated following the parameter name and a colon. At the present time, we only need basic numeric types such as the type `scala.Double` of double precision numbers. Scala defines type aliases for some standard types, so we can write numeric types as in Java. For instance `double` is a type alias of `scala.Double` and `int` is a type alias for `scala.Int`.
Functions with parameters are evaluated analogously to operators in expressions. First, the arguments of the function are evaluated (in left-to-right order). Then, the function application is replaced by the function's right hand side, and at the same time all formal parameters of the function are replaced by their corresponding actual arguments.

**Example**

      sumOfSquares(3, 2+2) → sumOfSquares(3, 4)
    → square(3) + square(4) → 3 * 3 + square(4)
    → 9 + square(4) →9+4*4
    → 9 + 16
    → 25

The example shows that the interpreter reduces function arguments to values before rewriting the function application. One could instead have chosen to apply the function to unreduced arguments. This would have yielded the following reduction sequence:

      sumOfSquares(3, 2+2)
    → square(3) + square(2+2) → 3 * 3 + square(2+2)
    → 9 + square(2+2)
    → 9 + (2+2) * (2+2)
    → 9+4*(2+2) →9+4*4
    → 9 + 16
    → 25
    
The second evaluation order is known as call-by-name, whereas the first one is known as call-by-value. For expressions that use only pure functions and that there- fore can be reduced with the substitution model, both schemes yield the same final values.
Call-by-value has the advantage that it avoids repeated evaluation of arguments. Call-by-name has the advantage that it avoids evaluation of arguments when the parameter is not used at all by the function. Call-by-value is usually more efficient than call-by-name, but a call-by-value evaluation might loop where a call-by-name evaluation would terminate. Consider:

    scala> def loop: Int = loop
    loop: Int

    scala> def first(x: Int, y: Int) = x
    first: (Int,Int)Int

Scala uses call-by-value by default, but it switches to call-by-name evaluation if the parameter type is preceded by `=>`.

**Example**

    scala> def constOne(x: Int, y: => Int) = 1
    constOne: (Int,=> Int)Int

    scala> constOne(1, loop)
    unnamed0: Int = 1

    scala> constOne(loop, 2) // gives an infinite loop.
    

## Conditional Expressions
Scala’s `if-else` lets one choose between two alternatives. Its syntax is like Java's `if-else`. But where Java's if-else can be used only as an alternative of statements, Scala allows the same syntax to choose between two expressions. That's why Scala's `if-else` serves also as a substitute for Java's conditional expression `... ? ... : ....`

***Example***
    
    scala> def abs(x: Double) = if (x >= 0) x else -x
    abs: (Double)Double

Scala's boolean expressions are similar to Java's; they are formed from the constants `true` and `false`, comparison operators, boolean negation `!` and the boolean operators `&&` and `||`.


# Scala Cheat Sheet


Contributed by Brendan O’Connor

http://docs.scala-lang.org/cheatsheets/

<html>

<table>
  <thead>
    <tr style="background-color: rgb(255, 255, 255);">
      <th>&nbsp;</th>
      <th>&nbsp;</th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><span id="variables" class="h2">variables</span></td>
      <td>&nbsp;</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><code class="highlighter-rouge">var x = 5</code></td>
      <td>variable</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><span style="background-color: green; color: white">GOOD</span> <code class="highlighter-rouge">val x = 5</code><br> <span style="background-color: red; color: white;">Bad</span> <code class="highlighter-rouge">x=6</code></td>
      <td>constant</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><code class="highlighter-rouge">var x: Double = 5</code></td>
      <td>explicit type</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><span id="functions" class="h2">functions</span></td>
      <td>&nbsp;</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><span style="background-color: green; color: white">GOOD</span> <code class="highlighter-rouge">def f(x: Int) = { x*x }</code><br> <span style="background-color: red; color: white">BAD</span> <code class="highlighter-rouge">def f(x: Int)   { x*x }</code></td>
      <td>define function <br> hidden error: without = it’s a Unit-returning procedure; causes havoc</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><span style="background-color: green; color: white">GOOD</span> <code class="highlighter-rouge">def f(x: Any) = println(x)</code><br> <span style="background-color: red; color: white">BAD</span> <code class="highlighter-rouge">def f(x) = println(x)</code></td>
      <td>define function <br> syntax error: need types for every arg.</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><code class="highlighter-rouge">type R = Double</code></td>
      <td>type alias</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><code class="highlighter-rouge">def f(x: R)</code> vs.<br> <code class="highlighter-rouge">def f(x: =&gt; R)</code></td>
      <td>call-by-value <br> call-by-name (lazy parameters)</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><code class="highlighter-rouge">(x:R) =&gt; x*x</code></td>
      <td>anonymous function</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><code class="highlighter-rouge">(1 to 5).map(_*2)</code> vs.<br> <code class="highlighter-rouge">(1 to 5).reduceLeft( _+_ )</code></td>
      <td>anonymous function: underscore is positionally matched arg.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">(1 to 5).map( x =&gt; x*x )</code></td>
      <td>anonymous function: to use an arg twice, have to name it.</td>
    </tr>
    <tr>
      <td><span style="background-color: green; color: white">GOOD</span> <code class="highlighter-rouge">(1 to 5).map(2*)</code><br> <span style="background-color: red; color: white">BAD</span> <code class="highlighter-rouge">(1 to 5).map(*2)</code></td>
      <td>anonymous function: bound infix method. Use <code class="highlighter-rouge">2*_</code> for sanity’s sake instead.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">(1 to 5).map { x =&gt; val y=x*2; println(y); y }</code></td>
      <td>anonymous function: block style returns last expression.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">(1 to 5) filter {_%2 == 0} map {_*2}</code></td>
      <td>anonymous functions: pipeline style. (or parens too).</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">def compose(g:R=&gt;R, h:R=&gt;R) = (x:R) =&gt; g(h(x))</code> <br> <code class="highlighter-rouge">val f = compose({_*2}, {_-1})</code></td>
      <td>anonymous functions: to pass in multiple blocks, need outer parens.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">val zscore = (mean:R, sd:R) =&gt; (x:R) =&gt; (x-mean)/sd</code></td>
      <td>currying, obvious syntax.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">def zscore(mean:R, sd:R) = (x:R) =&gt; (x-mean)/sd</code></td>
      <td>currying, obvious syntax</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">def zscore(mean:R, sd:R)(x:R) = (x-mean)/sd</code></td>
      <td>currying, sugar syntax. but then:</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">val normer = zscore(7, 0.4) _</code></td>
      <td>need trailing underscore to get the partial, only for the sugar version.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">def mapmake[T](g:T=&gt;T)(seq: List[T]) = seq.map(g)</code></td>
      <td>generic type.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">5.+(3); 5 + 3</code> <br> <code class="highlighter-rouge">(1 to 5) map (_*2)</code></td>
      <td>infix sugar.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">def sum(args: Int*) = args.reduceLeft(_+_)</code></td>
      <td>varargs.</td>
    </tr>
    <tr>
      <td><span id="packages" class="h2">packages</span></td>
      <td>&nbsp;</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">import scala.collection._</code></td>
      <td>wildcard import.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">import scala.collection.Vector</code> <br> <code class="highlighter-rouge">import scala.collection.{Vector, Sequence}</code></td>
      <td>selective import.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">import scala.collection.{Vector =&gt; Vec28}</code></td>
      <td>renaming import.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">import java.util.{Date =&gt; _, _}</code></td>
      <td>import all from java.util except Date.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">package pkg</code> <em>at start of file</em> <br> <code class="highlighter-rouge">package pkg { ... }</code></td>
      <td>declare a package.</td>
    </tr>
    <tr>
      <td><span id="data_structures" class="h2">data structures</span></td>
      <td>&nbsp;</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">(1,2,3)</code></td>
      <td>tuple literal. (<code class="highlighter-rouge">Tuple3</code>)</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">var (x,y,z) = (1,2,3)</code></td>
      <td>destructuring bind: tuple unpacking via pattern matching.</td>
    </tr>
    <tr>
      <td><span style="background-color: red; color: white">BAD</span><code class="highlighter-rouge">var x,y,z = (1,2,3)</code></td>
      <td>hidden error: each assigned to the entire tuple.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">var xs = List(1,2,3)</code></td>
      <td>list (immutable).</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">xs(2)</code></td>
      <td>paren indexing. (<a href="http://www.slideshare.net/Odersky/fosdem-2009-1013261/27">slides</a>)</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">1 :: List(2,3)</code></td>
      <td>cons.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">1 to 5</code> <em>same as</em> <code class="highlighter-rouge">1 until 6</code> <br> <code class="highlighter-rouge">1 to 10 by 2</code></td>
      <td>range sugar.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">()</code> <em>(empty parens)</em></td>
      <td>sole member of the Unit type (like C/Java void).</td>
    </tr>
    <tr>
      <td><span id="control_constructs" class="h2">control constructs</span></td>
      <td>&nbsp;</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">if (check) happy else sad</code></td>
      <td>conditional.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">if (check) happy</code> <em>same as</em> <br> <code class="highlighter-rouge">if (check) happy else ()</code></td>
      <td>conditional sugar.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">while (x &lt; 5) { println(x); x += 1}</code></td>
      <td>while loop.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">do { println(x); x += 1} while (x &lt; 5)</code></td>
      <td>do while loop.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">import scala.util.control.Breaks._</code><br><code class="highlighter-rouge">breakable {</code><br><code class="highlighter-rouge">    for (x &lt;- xs) {</code><br><code class="highlighter-rouge">        if (Math.random &lt; 0.1) break</code><br><code class="highlighter-rouge">    }</code><br><code class="highlighter-rouge">}</code></td>
      <td>break. (<a href="http://www.slideshare.net/Odersky/fosdem-2009-1013261/21">slides</a>)</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">for (x &lt;- xs if x%2 == 0) yield x*10</code> <em>same as</em> <br><code class="highlighter-rouge">xs.filter(_%2 == 0).map(_*10)</code></td>
      <td>for comprehension: filter/map</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">for ((x,y) &lt;- xs zip ys) yield x*y</code> <em>same as</em> <br><code class="highlighter-rouge">(xs zip ys) map { case (x,y) =&gt; x*y }</code></td>
      <td>for comprehension: destructuring bind</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">for (x &lt;- xs; y &lt;- ys) yield x*y</code> <em>same as</em> <br><code class="highlighter-rouge">xs flatMap {x =&gt; ys map {y =&gt; x*y}}</code></td>
      <td>for comprehension: cross product</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">for (x &lt;- xs; y &lt;- ys) {</code><br>    <code class="highlighter-rouge">println("%d/%d = %.1f".format(x,y, x*y))</code><br><code class="highlighter-rouge">}</code></td>
      <td>for comprehension: imperative-ish<br><a href="http://java.sun.com/javase/6/docs/api/java/util/Formatter.html#syntax">sprintf-style</a></td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">for (i &lt;- 1 to 5) {</code><br>    <code class="highlighter-rouge">println(i)</code><br><code class="highlighter-rouge">}</code></td>
      <td>for comprehension: iterate including the upper bound</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">for (i &lt;- 1 until 5) {</code><br>    <code class="highlighter-rouge">println(i)</code><br><code class="highlighter-rouge">}</code></td>
      <td>for comprehension: iterate omitting the upper bound</td>
    </tr>
    <tr>
      <td><span id="pattern_matching" class="h2">pattern matching</span></td>
      <td>&nbsp;</td>
    </tr>
    <tr>
      <td><span style="background-color: green; color: white">GOOD</span> <code class="highlighter-rouge">(xs zip ys) map { case (x,y) =&gt; x*y }</code><br> <span style="background-color: red; color: white">BAD</span> <code class="highlighter-rouge">(xs zip ys) map( (x,y) =&gt; x*y )</code></td>
      <td>use case in function args for pattern matching.</td>
    </tr>
    <tr>
      <td><span style="background-color: red; color: white">BAD</span><br><code class="highlighter-rouge">val v42 = 42</code><br><code class="highlighter-rouge">Some(3) match {</code><br><code class="highlighter-rouge">  case Some(v42) =&gt; println("42")</code><br><code class="highlighter-rouge">    case _ =&gt; println("Not 42")</code><br><code class="highlighter-rouge">}</code></td>
      <td>“v42” is interpreted as a name matching any Int value, and “42” is printed.</td>
    </tr>
    <tr>
      <td><span style="background-color: green; color: white">GOOD</span><br><code class="highlighter-rouge">val v42 = 42</code><br><code class="highlighter-rouge">Some(3) match {</code><br><code class="highlighter-rouge">   case Some(`v42`) =&gt; println("42")</code><br><code class="highlighter-rouge">case _ =&gt; println("Not 42")</code><br><code class="highlighter-rouge">}</code></td>
      <td>”`v42`” with backticks is interpreted as the existing val <code class="highlighter-rouge">v42</code>, and “Not 42” is printed.</td>
    </tr>
    <tr>
      <td><span style="background-color: green; color: white">GOOD</span><br><code class="highlighter-rouge">val UppercaseVal = 42</code><br><code class="highlighter-rouge">Some(3) match {</code><br><code class="highlighter-rouge">  case Some(UppercaseVal) =&gt; println("42")</code><br><code class="highlighter-rouge">    case _ =&gt; println("Not 42")</code><br><code class="highlighter-rouge">}</code></td>
      <td><code class="highlighter-rouge">UppercaseVal</code> is treated as an existing val, rather than a new pattern variable, because it starts with an uppercase letter. Thus, the value contained within <code class="highlighter-rouge">UppercaseVal</code> is checked against <code class="highlighter-rouge">3</code>, and “Not 42” is printed.</td>
    </tr>
    <tr>
      <td><span id="object_orientation" class="h2">object orientation</span></td>
      <td>&nbsp;</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">class C(x: R)</code> <em>same as</em> <br><code class="highlighter-rouge">class C(private val x: R)</code><br><code class="highlighter-rouge">var c = new C(4)</code></td>
      <td>constructor params - private</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">class C(val x: R)</code><br><code class="highlighter-rouge">var c = new C(4)</code><br><code class="highlighter-rouge">c.x</code></td>
      <td>constructor params - public</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">class C(var x: R) {</code><br><code class="highlighter-rouge">assert(x &gt; 0, "positive please")</code><br><code class="highlighter-rouge">var y = x</code><br><code class="highlighter-rouge">val readonly = 5</code><br><code class="highlighter-rouge">private var secret = 1</code><br><code class="highlighter-rouge">def this = this(42)</code><br><code class="highlighter-rouge">}</code></td>
      <td><br>constructor is class body<br>declare a public member<br>declare a gettable but not settable member<br>declare a private member<br>alternative constructor</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">new{ ... }</code></td>
      <td>anonymous class</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">abstract class D { ... }</code></td>
      <td>define an abstract class. (non-createable)</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">class C extends D { ... }</code></td>
      <td>define an inherited class.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">class D(var x: R)</code><br><code class="highlighter-rouge">class C(x: R) extends D(x)</code></td>
      <td>inheritance and constructor params. (wishlist: automatically pass-up params by default)</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">object O extends D { ... }</code></td>
      <td>define a singleton. (module-like)</td>
    </tr>
    <tr style="background-color: rgb(255, 255, 255);">
      <td><code class="highlighter-rouge">trait T { ... }</code><br><code class="highlighter-rouge">class C extends T { ... }</code><br><code class="highlighter-rouge">class C extends D with T { ... }</code></td>
      <td>traits.<br>interfaces-with-implementation. no constructor params. <a href="/tutorials/tour/mixin-class-composition.html">mixin-able</a>.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">trait T1; trait T2</code><br><code class="highlighter-rouge">class C extends T1 with T2</code><br><code class="highlighter-rouge">class C extends D with T1 with T2</code></td>
      <td>multiple traits.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">class C extends D { override def f = ...}</code></td>
      <td>must declare method overrides.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">new java.io.File("f")</code></td>
      <td>create object.</td>
    </tr>
    <tr>
      <td><span style="background-color: red; color: white">BAD</span> <code class="highlighter-rouge">new List[Int]</code><br> <span style="background-color: green; color: white">GOOD</span> <code class="highlighter-rouge">List(1,2,3)</code></td>
      <td>type error: abstract type<br>instead, convention: callable factory shadowing the type</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">classOf[String]</code></td>
      <td>class literal.</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">x.isInstanceOf[String]</code></td>
      <td>type check (runtime)</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">x.asInstanceOf[String]</code></td>
      <td>type cast (runtime)</td>
    </tr>
    <tr>
      <td><code class="highlighter-rouge">x: String</code></td>
      <td>ascription (compile time)</td>
    </tr>
  </tbody>
</table>

        </div>
        </html>